# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium

# Import API key
from api_keys import geoapify_key

In [14]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tazovsky,67.4667,78.7000,-4.20,93,100,5.67,RU,1728353875
1,1,nuuk,64.1835,-51.7216,2.34,69,18,5.14,GL,1728354469
2,2,utrik,11.2278,169.8474,28.69,76,1,3.91,MH,1728354470
3,3,waitangi,-43.9535,-176.5597,11.57,98,51,5.36,NZ,1728354471
4,4,sinabang,2.4803,96.3801,27.48,80,100,2.92,ID,1728354472


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Create a map centered around the average coordinates of the cities in city_data_df
map_center = [city_data_df["Lat"].mean(), city_data_df["Lng"].mean()]
m = folium.Map(location=map_center, zoom_start=2)

# Add circle markers for each city in the city_data_df DataFrame
for index, row in city_data_df.iterrows():
    folium.CircleMarker(
        location=[row["Lat"], row["Lng"]],
        radius=row["Humidity"] / 10,  # Adjust radius proportional to humidity
        popup=f'{row["City"]}, {row["Country"]}\nHumidity: {row["Humidity"]}%',
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.6
    ).add_to(m)

# Save the map as an HTML file
m.save("city_humidity_map.html")

# Display the map (only works in Jupyter Notebook)
m

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria
# For example, filtering cities with Max Temp > 20 and Humidity < 60
filtered_cities_df = city_data_df[(city_data_df["Max Temp"] > 20) & (city_data_df["Humidity"] < 60)]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,31,zaranj,30.9667,61.8833,20.19,41,0,6.43,AF,1728354506
41,41,bela vista,-22.1089,-56.5211,26.15,18,2,2.19,BR,1728354517
61,61,sparks,39.5349,-119.7527,20.48,35,0,4.63,US,1728354540
66,66,patrocinio,-18.9439,-46.9925,24.80,30,11,1.34,BR,1728354546
75,75,christchurch,-43.5333,172.6333,21.10,40,100,9.26,NZ,1728354556


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create a DataFrame called hotel_df to store city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tazovsky,RU,67.4667,78.7000,93,
1,nuuk,GL,64.1835,-51.7216,69,
2,utrik,MH,11.2278,169.8474,76,
3,waitangi,NZ,-43.9535,-176.5597,98,
4,sinabang,ID,2.4803,96.3801,80,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000  # Search radius in meters
params = {
    "categories": "accommodation.hotel",  # Category for hotels
    "limit": 1,                           # We want only the first hotel result
    "apiKey": geoapify_key                # Use the Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL for Geoapify Places API
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tazovsky - nearest hotel: Тазовчанка
nuuk - nearest hotel: Nordbo
utrik - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
sinabang - nearest hotel: No hotel found
uturoa - nearest hotel: Hawaiki Nui hotel
tchollire - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
yellowknife - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
anloga - nearest hotel: Hotel de White House
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
puerto natales - nearest hotel: Dorotea Patagonia Hostel
hamilton - nearest hotel: North Vista Manor
isafjordur - nearest hotel: Hótel Horn
peasedown saint john - nearest hotel: The Radstock Hotel
blackmans bay - nearest hotel: Villa Howden
port-aux-francais - nearest hotel: Keravel
cabedelo - nearest hotel: No hotel found
vepagunta - nearest hotel: OYO 60899 V3 Comforts
tralee - nearest hotel: The Ashe Hotel
colonia - nearest hotel: Wasserturm Hotel Co

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tazovsky,RU,67.4667,78.7000,93,Тазовчанка
1,nuuk,GL,64.1835,-51.7216,69,Nordbo
2,utrik,MH,11.2278,169.8474,76,No hotel found
3,waitangi,NZ,-43.9535,-176.5597,98,Hotel Chathams
4,sinabang,ID,2.4803,96.3801,80,No hotel found
...,...,...,...,...,...,...
561,nova vicosa,BR,-17.8919,-39.3719,81,Pousada Pontal da Barra
562,lithgow,AU,-33.4833,150.1500,85,Grand Central Hotel
563,biltine,TD,14.5333,20.9167,48,فندق ثلاث نجوم بلتن
564,aginskoye,RU,51.1000,114.5300,30,Алтан Сэргэ


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
# Create a map centered around the average coordinates of the cities in hotel_df
map_center = [hotel_df["Lat"].mean(), hotel_df["Lng"].mean()]
m = folium.Map(location=map_center, zoom_start=2)

# Add circle markers for each city in the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Create a popup message that includes city, country, humidity, and hotel name
    popup_message = f"City: {row['City']}<br>Country: {row['Country']}<br>Humidity: {row['Humidity']}%<br>Hotel: {row['Hotel Name']}"

    # Add a circle marker to the map
    folium.CircleMarker(
        location=[row["Lat"], row["Lng"]],
        radius=row["Humidity"] / 10,  # Adjust radius proportional to humidity
        popup=folium.Popup(popup_message, max_width=300),  # Popup with the city, country, and hotel name
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.6
    ).add_to(m)

# Save the map as an HTML file
m.save("hotel_humidity_map.html")

# Display the map in the notebook (only works in Jupyter Notebook)
m